#### Bllossom Llama2 13bhf 4bit 양자화 모델 추론 
```
1. excute: github:https://huggingface.co/ybjeong/bllossom_llama-2-13b-chat_ko_0725-4bit
2. parameter data type: touch.float16
3. qutanization: 8 bit quant
4. processor: cpu
5. write type: pytorch bin
6. transformers: AutoModelForCausalLM, pipeline
```

In [ ]:
import torch
from transformers import AutoModelForCausalLM, pipeline

In [ ]:
# bllossom_llama2_13bchat 4BIT 양자화 모델
# 파일형식: PyTouch 모델 저장 방식 (확장자: bin)
# 파일크기: 9.3G 2개, 5.8G 1개
model_id = "/data/bwllm/models/bllossom-llama2-13bhf-chat-4bit"

In [ ]:
model_id = "/data/bwllm/models/casllm-base-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype=torch.float16, device_map="auto")

In [ ]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

In [ ]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

In [ ]:
print(model)

In [ ]:
# 모델의 모든 파라미터의 데이터 타입 확인
for name, param in model.named_parameters():
    print(f"{name} data type: {param.dtype}")

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    #device=0,
)

In [ ]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text

In [ ]:
# 문장 생성
def generate(prompt_text):
    answer = pipe(
        prompt_text,
        do_sample=True,
        max_new_tokens=2000,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    return answer[0]['generated_text']

In [ ]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템 감리 절차 과정에 대해 설명해줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

In [ ]:
# 답변 텍스트만 출력
answer = generate(prompt_text)
print(answer)